In [1]:
from unityagents import UnityEnvironment
from collections import deque

import numpy as np
import progressbar as pb

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable

In [2]:
# 1 agent
# env = UnityEnvironment(file_name="./Reacher_Windows_x86_64_1_agent/Reacher.exe")

# 20 agents
env = UnityEnvironment(file_name="./Reacher_Windows_x86_64_20_agents/Reacher.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
pi = Variable(torch.FloatTensor([math.pi]), requires_grad=True).cuda()

def normal(x, mu, sigma_sq):
    a = (-1*(Variable(x, requires_grad=True)-mu).pow(2)/(2*sigma_sq)).exp()
    b = 1/(2*sigma_sq*pi.expand_as(sigma_sq)).sqrt()
    return a*b

def probs_to_actions(mu, sigma_sq):
    eps = torch.randn(mu.size())
    sigma_sq = torch.ones(mu.size()).cuda()
    action = (mu + sigma_sq.sqrt()*Variable(eps, requires_grad=True).cuda())
    prob = normal(action, mu, sigma_sq)
    
    return action.reshape(-1, 4), torch.log(prob).reshape(-1, 4)

In [7]:
class Policy(nn.Module):
    def __init__(self, state_size, action_size, n_agents=1, fc1_size=128, fc2_size=64, fc3_size=64):
        super(Policy, self).__init__()
        
        self.fc1 = nn.Linear(state_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.fc3 = nn.Linear(fc2_size, fc3_size)
        
        self.fc4_mu = nn.Linear(fc3_size, action_size)
        self.fc4_sigma_squared = nn.Linear(fc3_size, action_size)
        
        self.n_agents = n_agents
        
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))

        mu = torch.tanh(self.fc4_mu(x))
        sigma = torch.sigmoid(self.fc4_sigma_squared(x))
        
        return mu, sigma
    
    def act(self, state):
        if not isinstance(state, torch.Tensor):
            state = torch.from_numpy(state).float().to(device)
        mu, sigma = self.forward(state)
        return probs_to_actions(mu, sigma)

In [8]:
class Value(nn.Module):
    """
        Critic network that estimates a Value Function used as a baseline
    """
    def __init__(self, state_size, action_size=1, n_agents=1, fc1_size=128, fc2_size=64, fc3_size=64):
        super(Value, self).__init__()
        
        self.fc1 = nn.Linear(state_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.fc3 = nn.Linear(fc2_size, fc3_size)
        self.fc4 = nn.Linear(fc3_size, action_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x)
    
    def estimate(self, state):
        state = torch.from_numpy(state).float().to(device)
        return self.forward(state)

In [9]:
def unroll_trajectory(env, brain_name, n_agents, t_max=1000, gamma=1.0):
    """
        Unroll a trajectory and return 
        list of action probabilities, states, actions and rewards

        Parameters
        ----------
        env: unityagents.UnityEnvironment
        Environment to play on.
        brain_name: String
        Name of brain used for UnityEnvironment
        policy: torch.nn.Module
        A neural network mapping states to action probabilities 
        The action to take at a given state
        t_max: int
        Maximum number of episodes in trajectory
    """
    
    env_info = env.reset(train_mode=True)[brain_name]
    # num_agents = len(env.ps)

    state_list      = []
    reward_list     = []
    prob_list       = []
    action_list     = []
    values_list     = []
    advantages_list = []

    states = np.array(env_info.vector_observations).reshape(n_agents, -1)
    
    for _ in range(t_max):

        a, p = policy_network.act(states)
        actions = a.cpu().detach().numpy()
        action_probabilities = p
        
        env_info = env.step(actions)[brain_name]
        next_states = np.array(env_info.vector_observations).reshape(n_agents, -1)
        
        prob_list.append(action_probabilities)
        state_list.append(states)
        action_list.append(actions)
        
        rewards = torch.from_numpy(np.array(env_info.rewards).reshape(n_agents, -1)).float().to(device)
        reward_list.append(rewards)
        
        cur_value    = value_network.estimate(states)
        next_value   = value_network.estimate(next_states)
        
        values       = rewards + gamma * cur_value
        values_list.append(values)
        advantages   = rewards + gamma * next_value - cur_value
        advantages_list.append(advantages)
        
        if np.any(env_info.local_done):
            break

        states = next_states

    return {
        "log_probs"  : torch.cat(prob_list).reshape(n_agents, t_max, -1),
        "states"     : np.array(state_list).reshape(n_agents, t_max, -1), 
        "actions"    : np.array(action_list).reshape(n_agents, t_max, -1),
        "rewards"    : torch.cat(reward_list).reshape(n_agents, t_max), 
        "values"     : torch.cat(values_list).reshape(n_agents, t_max), 
        "advantages" : torch.cat(advantages_list).reshape(n_agents, t_max),
    }

In [10]:
def ppo_loss(old_probs, states, R, epsilon=0.3):
    """
        Loss as defined in PPO algorithm

        Parameters
        ----------
        old_probs: torch.Tensor
        Tensor that defines log probabilities of actions under evaluated policy
        states: np.Array [n_agents x n_episoded x state_size] 
        signal: np.Array [n_agents x n_episoded x state_size] 
    """
    old_probs = torch.tensor(old_probs, dtype=torch.float, device=device)
    _, new_probs = policy_network.act(states)
    new_probs = new_probs.reshape(old_probs.shape)        
    ratio = (new_probs/old_probs)    
    clip = torch.clamp(ratio, 1-epsilon, 1+epsilon)
    return (-torch.min(ratio, clip)  * torch.mean(R)).sum()

In [11]:
def reinforce_loss(log_probs, R):
    """
        Loss as defined in REINFORCE algorithm

        Parameters
        ----------
        log_probs: torch.Tensor
        Tensor that defines compute graph for action log probabilities
        R: torch.Tensor
        Tensor that defines rewards along each episode in trajectory
    """
    R = torch.from_numpy(R).float().to(device)
    return (-log_probs * torch.mean(R)).sum()

In [12]:
def a3c(n_episodes=1000, max_t=2000, print_every=100, n_pp_iterations=3, flavor='PPO'):
    widget = ['training loop: ', pb.Percentage(), ' ', pb.Bar(), ' ', pb.ETA() ]
    timer = pb.ProgressBar(widgets=widget, maxval=n_episodes).start()
    
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        
        trajectory = unroll_trajectory(env, brain_name, num_agents, max_t)

        mean_rewards = trajectory['rewards'].sum(1).mean().detach().cpu().numpy()
        scores_deque.append(mean_rewards)
        scores.append(mean_rewards)
        
        # Modification 2: Use Future Rewards
        R = trajectory['rewards'].detach().cpu().numpy()
        R = np.flip(np.flip(R).cumsum(1)).reshape(num_agents, max_t).copy()
        A = trajectory['advantages'].detach().cpu().numpy().reshape(R.shape)
                
        # Modification 1: Batch Normalization of Reward Signal
        if R.shape[0] > 10:
            R_mean = np.mean(R, axis=0)
            R_std = np.std(R, axis=0)
            R = np.nan_to_num((R - R_mean) / R_std)
            
        if A.shape[0] > 10:
            A_mean = np.mean(A, axis=0)
            A_std = np.std(A, axis=0)
            A = np.nan_to_num((A - A_mean) / A_std)
            
        if flavor == 'REINFORCE':
            policy_loss = reinforce_loss(trajectory['log_probs'], R)
            policy_optimizer.zero_grad()
            policy_loss.backward()
            policy_optimizer.step()
            
        if flavor == "PPO":
            states = torch.from_numpy(trajectory['states']).float().to(device)
            value_loss = value_criterion(
                value_network(states).reshape(num_agents, max_t), 
                trajectory['values']
            )
            value_optimizer.zero_grad()
            value_loss.backward()
            value_optimizer.step()
            
            signal = torch.from_numpy((R-A)).float().to(device)
            for i in range(n_pp_iterations):
                policy_loss = ppo_loss(trajectory['log_probs'], states, signal)
                policy_optimizer.zero_grad()
                policy_loss.backward()
                policy_optimizer.step()

        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        
        if np.mean(scores_deque)>=30.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
            
        timer.update(i_episode)
        
    return scores

In [ ]:
policy_network   = Policy(state_size=state_size, action_size=action_size).to(device)
policy_optimizer = optim.Adam(policy_network.parameters(), lr=1e-4)

value_network    = Value(state_size=state_size, action_size=1).to(device)
value_criterion  = nn.MSELoss()
value_optimizer  = optim.Adam(value_network.parameters(), lr=1e-4)

%time scores = a3c(n_episodes=1000, max_t=1000, print_every=100)

D:\Anaconda3\envs\gym\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide
D:\Anaconda3\envs\gym\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()